In [2]:
from main.models import Product, SaleRecord, Channel
from main.display import table
from django.db.models import Subquery, Max, OuterRef, F, Sum


## RECAP

```python
print queryset.query  ## get raw sql


```

## 最新成果

In [35]:
p1 = Product.objects.first()
p1_record = SaleRecord.objects.filter(product=p1)

newest = p1_record.order_by('datetime').reverse()

print p1.name
print newest.first().datetime
print newest.first().price
print newest.first().count

王紫麵
2018-07-06 00:00:00+00:00
18
64


In [99]:
sales_record_queryset = SaleRecord.objects.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost')
)

sales_record_queryset = sales_record_queryset.filter(
    product=OuterRef('pk')
)

latest_record = sales_record_queryset.order_by('datetime').reverse()[:1]

products = Product.objects.all().values(
    _product_name=F('name'),
    unit_cost=F('cost'),
    latest_amount=Subquery(latest_record.values('amount')),
    latest_margin=Subquery(latest_record.values('margin')),
    latest_cost=Subquery(latest_record.values('cost')),
    latest_count=Subquery(latest_record.values('count')),
    latest_price=Subquery(latest_record.values('price')),
    channel=Subquery(latest_record.values('channel__name')),
)

table(products)  ## only for show up

,_product_name,channel,latest_amount,latest_cost,latest_count,latest_margin,latest_price,unit_cost
0,王紫麵,加熱福,1152,512,64,640,18,8
1,真由味,加熱福,8092,5202,289,2890,28,18
2,湖遲屋,加熱福,4248,3068,118,1180,36,26
3,品嗑,加熱福,14256,11286,297,2970,48,38


## 今年總營業額

In [58]:

sale_queryset = SaleRecord.objects.all()

sale_queryset = sale_queryset.filter(datetime__gte='2018-01-01')

# from django.db.models.functions import Now, TruncYear
# sale_queryset = sale_queryset.annotate(year=TruncYear('datetime'))
# sale_queryset = sale_queryset.filter(year__gte=TruncYear(Now()))

sale_queryset = sale_queryset.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost')
)

sale_queryset.aggregate(
    total_amount=Sum('amount'),
    total_margin=Sum('margin'),
    total_cost=Sum('cost')
)



{'total_amount': 174869, 'total_cost': 137168, 'total_margin': 37701}

### 每一季的營業額

In [51]:
from datetime import datetime
from django.db.models.functions import Trunc

sale_questset = SaleRecord.objects.all()

sale_questset = sale_questset.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost'),
    month=Trunc('datetime', 'month'),
)

sale_questset = sale_questset.annotate(
    total_amount=Sum('amount'),
    total_cost=Sum('cost'),
    total_margin=Sum('margin'),
)

sale_questset = sale_questset.values(
    'product__name', 
    'month', 
    'total_margin',
    'total_amount',
    'total_cost',
).annotate(sale_count=Sum('count'))


table(sale_questset)

,month,product__name,sale_count,total_amount,total_cost,total_margin
0,2017-07-01 00:00:00+00:00,品嗑,335,16153,12730,3423
1,2017-10-01 00:00:00+00:00,品嗑,374,16702,14212,2490
2,2018-01-01 00:00:00+00:00,品嗑,722,33921,27436,6485
3,2018-04-01 00:00:00+00:00,品嗑,544,24662,20672,3990
4,2018-07-01 00:00:00+00:00,品嗑,524,23794,19912,3882
5,2017-07-01 00:00:00+00:00,湖遲屋,486,16576,12636,3940
6,2017-10-01 00:00:00+00:00,湖遲屋,514,17048,13364,3684
7,2018-01-01 00:00:00+00:00,湖遲屋,449,15020,11674,3346
8,2018-04-01 00:00:00+00:00,湖遲屋,631,22244,16406,5838
9,2018-07-01 00:00:00+00:00,湖遲屋,262,7952,6812,1140


## 各個 channel 總成績


In [90]:

sale_queryset = SaleRecord.objects.all()

channel = Channel.objects.first()

# sale_queryset = sale_queryset.filter(channel=channel)

sale_queryset = sale_queryset.filter(channel=OuterRef('pk'))

sale_queryset = sale_queryset.annotate(
    amount=F('price')*F('count'),
    cost=F('product__cost')*F('count')
).annotate(
    margin=F('amount') - F('cost')
)

sale_queryset = sale_queryset.annotate(
    total_amount=Sum('amount'),
    total_cost=Sum('cost'),
    total_margin=Sum('margin'),
)

sale_queryset = sale_queryset.values(
    'channel',
    'total_amount',
    'total_cost',
    'total_margin',
).annotate(sale_count=Sum('count'))


channel_queryset = Channel.objects.all().annotate(
    
    total_amount=Subquery(sale_queryset.values('total_amount')),
    total_margin=Subquery(sale_queryset.values('total_margin')),
    total_cost=Subquery(sale_queryset.values('total_cost')),

)

from django.db.models import ExpressionWrapper, FloatField
from django.db.models.functions import Substr

rate_expression = Substr(((F('total_margin')+0.0)/F('total_amount')*100), 1, 2)

channel_queryset = channel_queryset.annotate(
    
    profit_rate= rate_expression,
)

table(channel_queryset.values())

,name,profit_rate,total_amount,total_cost,total_margin
0,加熱福,28,102976,73456,29520
1,好是多,6.,84800,79432,5368
2,拳家,27,81351,58592,22759
